TensorFlow was designed to be flexible, scalable, and production-ready, and existing frameworks
arguably hit only two out of the three of these. Here are some of TensorFlow’s highlights:

- It runs not only on Windows, Linux, and macOS, but also on mobile devices,including both iOS and Android.


- It provides a very simple Python API called TF.Learn2 (tensorflow.contrib.learn), compatible with Scikit-Learn. As you will see, you can use it to train various types of neural networks in just a few lines of code. It was previously an independent project called Scikit Flow (or skflow).


- It also provides another simple API called TF-slim (tensorflow.contrib.slim)to simplify building, training, and evaluating neural networks.


- Several other high-level APIs have been built independently on top of Tensor‐Flow, such as Keras or Pretty Tensor.


- Its main Python API offers much more flexibility (at the cost of higher complexity)to create all sorts of computations, including any neural network architecture you can think of.


- It includes highly efficient C++ implementations of many ML operations, particularly those needed to build neural networks. There is also a C++ API to define your own high-performance operations.


- It provides several advanced optimization nodes to search for the parameters that minimize a cost function. These are very easy to use since TensorFlow automatically takes care of computing the gradients of the functions you define. This is called automatic differentiating (or autodiff).


- It also comes with a great visualization tool called TensorBoard that allows you to browse through the computation graph, view learning curves, and more.


-  Google also launched a cloud service to run TensorFlow graphs.


- Last but not least, it has a dedicated team of passionate and helpful developers,and a growing community contributing to improving it. It is one of the most popular open source projects on GitHub, and more and more great projects are being built on top of it (for examples, check out the resources page on https://www.tensorflow.org/, or https://github.com/jtoy/awesome-tensorflow). To ask technical questions, you should use http://stackoverflow.com/ and tag your question with "tensorflow". You can file bugs and feature requests through GitHub.For general discussions, join the Google group.

**Figure 9-1. A simple computation graph**

<img align="left" src="figures/figure9-1.png">

**Figure 9-2. Parallel computation on multiple CPUs/GPUs/servers**

<img align="left" src="figures/figure9-2.png">

## Creating Your First Graph and Running It in a Session

The following code creates the graph represented in Figure 9-1:

In [1]:
import tensorflow as tf

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x*x*y + y + 2  # It just creates a computation graph

It just creates a computation graph. In fact, even the variables are not initialized
yet. To evaluate this graph, you need to open a TensorFlow session and use it
to initialize the variables and evaluate $f$.

A TensorFlow session takes care of placing the operations onto devices such as CPUs and GPUs and running them, and it holds all the variable values. The following code creates a session, initializes the variables,
and evaluates, and $f$ then closes the session (which frees up resources):

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)

sess.close()

42


Having to repeat ``sess.run()`` all the time is a bit cumbersome, but fortunately there is a better way:

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


Inside the with block, the session is set as the default session. Calling ``x.initializer.run()`` is equivalent to calling ``tf.get_default_session().run(x.initializer)``, and similarly ``f.eval()`` is equivalent to calling ``tf.get_default_session().run(f)``. This makes the code easier to read. Moreover,the session is automatically closed at the end of the block.

Instead of manually running the initializer for every single variable, you can use the ``global_variables_initializer()`` function. Note that it does not actually perform the initialization immediately, but rather creates a node in the graph that will initialize all variables when it is run:

In [7]:
init = tf.global_variables_initializer()  #  prepare an init node

with tf.Session() as sess:
    init.run()                           #  actually initialize all the variables
    result = f.eval()
    print(result)

42


A TensorFlow program is typically split into two parts: the first part builds a computation
graph (this is called the **construction phase**), and the second part runs it (this is
the **execution phase**). The construction phase typically builds a computation graph
representing the ML model and the computations required to train it. The execution
phase generally runs a loop that evaluates a training step repeatedly (for example, one
step per mini-batch), gradually improving the model parameters. We will go through
an example shortly.

## Managing Graphs
Any node you create is automatically added to the default graph:

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In most cases this is fine, but sometimes you may want to manage multiple independent
graphs. You can do this by creating a new Graph and temporarily making it the
default graph inside a with block, like so:

In [9]:
graph = tf.Graph()

In [13]:
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph,   x2.graph is tf.get_default_graph()

(True, False)

```
In Jupyter (or in a Python shell), it is common to run the same commands more than once while 
you are experimenting. As a result, you may end up with a default graph containing many
duplicate nodes. One solution is to restart the Jupyter kernel (or the Python shell), but a 
more convenient solution is to just reset the default graph by running tf.reset_default_graph().
```

## Linear Regression with TensorFlow

TensorFlow operations (also called ops for short) can take any number of inputs and
produce any number of outputs.